In [3]:
import numpy as np
from typing import List, Tuple, Dict, Any


def print_interval_table(header: str, rows: List[str],
                         data: List[List[Tuple[float, float]]]):
    """
    Печать таблицы интервалов.
    rows: имена строк (A1, A2, ...)
    data: список строк, каждая строка – список интервалов (L,U)
    """
    print("\n" + header)
    # заголовок: критерии C1, C2, ...
    n = len(data[0])
    cols = [f"C{j+1}" for j in range(n)]
    print("{:>6} ".format(""), end="")
    for c in cols:
        print("{:^24}".format(c), end="")
    print()
    for name, row in zip(rows, data):
        print("{:>6} ".format(name), end="")
        for (L, U) in row:
            print("[{:8.4f}, {:8.4f}] ".format(L, U), end="")
        print()


def print_scalar_table(header: str, rows: List[str],
                       cols: List[str], data: np.ndarray):
    """
    Печать таблицы чисел (PIS/NIS и т.п.)
    data: shape (len(rows), len(cols))
    """
    print("\n" + header)
    print("{:>6} ".format(""), end="")
    for c in cols:
        print("{:^12}".format(c), end="")
    print()
    for i, name in enumerate(rows):
        print("{:>6} ".format(name), end="")
        for j in range(len(cols)):
            print("{:12.4f}".format(data[i, j]), end="")
        print()


def compare_intervals(a: Tuple[float, float],
                      b: Tuple[float, float],
                      alpha: float) -> int:
    """
    Сравнение интервалов [aL,aU], [bL,bU] по правилу из статьи.
    -1: a < b, 0: a ~ b, 1: a > b.
    """
    aL, aU = a
    bL, bU = b

    # (1) нет пересечения
    if aU <= bL:
        return -1
    if bU <= aL:
        return 1

    # (2) совпадают
    if abs(aL - bL) < 1e-12 and abs(aU - bU) < 1e-12:
        return 0

    # (3) aL ≤ bL < bU ≤ aU
    if aL <= bL < bU <= aU:
        if alpha * (bL - aL) >= (1 - alpha) * (aU - bU):
            return -1
        else:
            return 1

    # (4) aL < bL < aU < bU
    if aL < bL < aU < bU:
        if alpha * (bL - aL) >= (1 - alpha) * (bU - aU):
            return -1
        else:
            return 1

    ca = 0.5 * (aL + aU)
    cb = 0.5 * (bL + bU)
    return -1 if ca < cb else (1 if ca > cb else 0)

def interval_vikor(
    matrix: np.ndarray,   # (m, n, 2): [L, U]
    weights: List[float], # len n
    types: List[int],     # 1=benefit, -1=cost
    v: float = 0.5,
    alpha: float = 0.5
) -> Dict[str, Any]:
    m, n = matrix.shape[0], matrix.shape[1]
    L = matrix[:, :, 0]
    U = matrix[:, :, 1]

    alt_names = [f"A{i+1}" for i in range(m)]
    crit_names = [f"C{j+1}" for j in range(n)]

    w = np.array(weights, dtype=float)
    w = w / w.sum()

    interval_matrix = [[(L[i, j], U[i, j]) for j in range(n)] for i in range(m)]
    print_interval_table("Таблица 1 - Матрица интервалов", alt_names, interval_matrix)

    f_star = np.zeros(n)   
    f_minus = np.zeros(n)  

    for j in range(n):
        if types[j] == 1:  # benefit
            f_star[j] = np.max(U[:, j])
            f_minus[j] = np.min(L[:, j])
        else:              # cost
            f_star[j] = np.min(L[:, j])
            f_minus[j] = np.max(U[:, j])

    pis_nis = np.vstack([f_star, f_minus])  # 2 x n
    print_scalar_table("Таблица 2 - f* и f-", ["f*", "f-"], crit_names, pis_nis)

    S_L = np.zeros(m)
    S_U = np.zeros(m)
    R_L = np.zeros(m)
    R_U = np.zeros(m)

    for i in range(m):
        sL = 0.0
        sU = 0.0
        rL_terms = []
        rU_terms = []
        for j in range(n):
            denom = f_star[j] - f_minus[j]
            if abs(denom) < 1e-12:
                continue
            if types[j] == 1:  # benefit
                term_L = w[j] * (f_star[j] - U[i, j]) / denom
                term_U = w[j] * (f_star[j] - L[i, j]) / denom
            else:              # cost
                term_L = w[j] * (L[i, j] - f_minus[j]) / denom
                term_U = w[j] * (U[i, j] - f_minus[j]) / denom
            sL += term_L
            sU += term_U
            rL_terms.append(term_L)
            rU_terms.append(term_U)
        S_L[i] = sL
        S_U[i] = sU
        R_L[i] = max(rL_terms) if rL_terms else 0.0
        R_U[i] = max(rU_terms) if rU_terms else 0.0

    SR_intervals = [[(S_L[i], S_U[i]), (R_L[i], R_U[i])] for i in range(m)]
    print_interval_table("Таблица 3 - S и R",
                         alt_names, SR_intervals)

    S_star = np.min(S_L)
    S_minus = np.max(S_U)
    R_star = np.min(R_L)
    R_minus = np.max(R_U)

    def safe_div(num, den):
        return num / den if abs(den) > 1e-12 else 0.0

    Q_L = np.zeros(m)
    Q_U = np.zeros(m)
    for i in range(m):
        Q_L[i] = (
            v * safe_div(S_L[i] - S_star, S_minus - S_star)
            + (1 - v) * safe_div(R_L[i] - R_star, R_minus - R_star)
        )
        Q_U[i] = (
            v * safe_div(S_U[i] - S_star, S_minus - S_star)
            + (1 - v) * safe_div(R_U[i] - R_star, R_minus - R_star)
        )

    Q_intervals = [[(Q_L[i], Q_U[i])] for i in range(m)]
    print_interval_table("Таблица 4 - Q",
                         alt_names, Q_intervals)


    # Итоговое ранжирование по Q с учётом alpha
    intervals_Q = [(Q_L[i], Q_U[i]) for i in range(m)]
    idxs = list(range(m))
    changed = True
    while changed:
        changed = False
        for k in range(len(idxs) - 1):
            i, j = idxs[k], idxs[k + 1]
            # минимальный Q – лучший, поэтому если i > j, меняем местами
            if compare_intervals(intervals_Q[i], intervals_Q[j], alpha) > 0:
                idxs[k], idxs[k + 1] = idxs[k + 1], idxs[k]
                changed = True

    ranking = idxs  # индексы альтернатив от лучшей к худшей

    print("Ранжирование:")
    print(" > ".join(alt_names[i] for i in ranking))

    return {
        "S": list(zip(S_L.tolist(), S_U.tolist())),
        "R": list(zip(R_L.tolist(), R_U.tolist())),
        "Q": list(zip(Q_L.tolist(), Q_U.tolist())),
        "f_star": f_star.tolist(),
        "f_minus": f_minus.tolist(),
        "ranking": ranking,
    }


if __name__ == "__main__":
    # Пример из статьи - Extension of VIKOR method for decision making problem with interval numbers
    matrix = np.array([
        [[0.75, 1.24], [2784, 3192]],  # A1
        [[1.83, 2.11], [3671, 3857]],  # A2
        [[4.90, 5.73], [4409, 4681]],  # A3
    ])
    weights = [1, 1]
    types = [-1, 1]      # C1 = мин, C2 = макс
    res = interval_vikor(matrix, weights, types, v=0.5, alpha=0.6)



Таблица 1 - Матрица интервалов
                  C1                      C2           
    A1 [  0.7500,   1.2400] [2784.0000, 3192.0000] 
    A2 [  1.8300,   2.1100] [3671.0000, 3857.0000] 
    A3 [  4.9000,   5.7300] [4409.0000, 4681.0000] 

Таблица 2 - f* и f-
            C1          C2     
    f*       0.7500   4681.0000
    f-       5.7300   2784.0000

Таблица 3 - S и R
                  C1                      C2           
    A1 [  0.8925,   0.9508] [  0.5000,   0.5000] 
    A2 [  0.6088,   0.6297] [  0.3916,   0.3635] 
    A3 [  0.0833,   0.0717] [  0.0833,   0.0717] 

Таблица 4 - Q
                  C1           
    A1 [  0.9664,   1.0000] 
    A2 [  0.6727,   0.6510] 
    A3 [  0.0000,  -0.0207] 
Ранжирование:
A3 > A2 > A1
